In [ ]:
import numpy as np
from sklearn import svm
from sklearn.datasets import make_blobs
import plotly.express as px
import plotly.graph_objects as go
import math
from dataclasses import dataclass

# defining plotting class etc

In [ ]:
@dataclass
class Plane:
    a: float
    b: float
    c: float
    d: float
    
    @property
    def normal(self):
        return np.array([self.a, self.b, self.c])
    
    def __contains__(self, point:np.array):
        return np.isclose(self.a*point[0] + self.b*point[1] + self.c*point[2] + self.d, 0)
    
    def project(self, point):
        x,y,z = point
        k = (self.a*x + self.b*y + self.c*z + self.d)/(self.a**2 + self.b**2 + self.c**2)
        return np.array([x - k*self.a, y-k*self.b, z-k*self.c])
    
    def z(self, x, y):
        return (- self.d - self.b*y - self.a*x)/self.c
    

def normalize(vec):
    return vec/np.linalg.norm(vec)

def make_base_changer(plane):
    uvec1 = normalize(plane.normal)
    uvec2 = normalize([0, -plane.d/plane.b, plane.d/plane.c])
    uvec3 = normalize(np.cross(uvec1, uvec2))
    transition_matrix = np.array([uvec1, uvec2, uvec3]).T
    
    origin = np.array([0,0,0])
    new_origin = plane.project(origin)
    origin_in_new_base = - transition_matrix.dot(new_origin)
    forward = lambda point: np.linalg.inv(transition_matrix).dot(point - origin_in_new_base) 
    backward  = lambda point: transition_matrix.dot(point) + origin_in_new_base
    return forward, backward


In [ ]:
class ThreeDFigure():
    def __init__(self, trafo_fn=None, back_trafo_fn=None):
        self.min_x = self.max_x = self.min_y = self.max_y = 0
        self.trafo_fn = trafo_fn if trafo_fn is not None else lambda x: x
        self.back_trafo_fn = back_trafo_fn if back_trafo_fn is not None else lambda x: x
        #https://community.plotly.com/t/creating-a-3d-scatterplot-with-equal-scale-along-all-axes/15108/7
        self.fig = go.Figure(layout=go.Layout(
                        scene=dict(camera=dict(eye=dict(x=1, y=1, z=1)), aspectmode="data"),
                        autosize=True,
                        width=1000,
                        height=800,
                        margin=dict(l=10, r=10, b=10, t=10, pad=4),
                        paper_bgcolor="White"))
        
    def _transform(self, points, inverse=False):
        trafo_fn = self.back_trafo_fn if inverse else self.trafo_fn              
        points = np.array([trafo_fn(point) for point in points])        
        return points
                    
    def add_surface(self, x, y, z_func):
        xy_arr = np.vstack([xx.flatten(), yy.flatten()]).T
        z_arr = np.array([z_func(*i) for i in xy_arr])
        points = np.column_stack([xy_arr, z_arr])
        points = self._transform(points)
        #points = np.column_stack([points[:,:2], np.array([0 for i in xy_arr])])
        surface_form = lambda x: x.reshape(round(math.sqrt(x.shape[0])),-1)
        self.fig.add_trace(go.Surface(x=surface_form(points[:,0]), y=surface_form(points[:,1]), z=surface_form(points[:,2])))
    
    
    def add_plane(self, plane):
        points = [[x, y, plane.z(x,y)] for x in [self.min_x, self.max_x] for y in [self.min_y, self.max_y]]
        points = self._transform(points)
        self.add_line(points[0], points[1], do_transform=False)
        self.add_line(points[1], points[2], do_transform=False)
        self.add_line(points[2], points[3], do_transform=False)
        self.add_line(points[3], points[0], do_transform=False)
        self.add_line(points[0], points[2], do_transform=False)
        self.add_line(points[1], points[3], do_transform=False)
        
    
    def add_line(self, point1, point2, width=6, do_transform=True):
        if do_transform:
            point1 = self._transform(np.array([point1])).squeeze()
            point2 = self._transform(np.array([point2])).squeeze()
        self.fig.add_trace(
            go.Scatter3d(x = [point1[0], point2[0]],
                         y = [point1[1], point2[1]],
                         z = [point1[2], point2[2]],
                         marker = dict(size = 1),
                         line = dict(width = width)
                         )
        )
        
    def add_markers(self, points, color="black", size=2):
        points = np.array(points)
        if points.ndim == 1: points = np.array([points])
        points = self._transform(points)
        
        self.min_x = min(self.min_x, min(points[:,0]))
        self.max_x = max(self.max_x, max(points[:,0]))
        self.min_y = min(self.min_y, min(points[:,1]))
        self.max_y = max(self.max_y, max(points[:,1]))
        
        self.fig.add_trace(
            go.Scatter3d(
                mode='markers',
                x=points[:,0],
                y=points[:,1],
                z=points[:,2],
                marker={"color": color,
                        "size": size,
                        "line": {"width": 0}
                       },
            )
        )
        
    def show(self):
        return self.fig

def make_meshgrid(val=0.1, amount=30):
    lsx = np.linspace(-val, val, amount)
    lsy = np.linspace(-val, val, amount)
    xx, yy = np.meshgrid(lsx,lsy)
    return xx, yy

In [ ]:
X, y = make_blobs(n_samples=80, centers=2, random_state=2, n_features=3)
model = svm.LinearSVC(C=1, max_iter=10000).fit(X, y)
print("Coefficients:", model.coef_, "Intercept:", model.intercept_)
decision_plane = Plane(*model.coef_[0], model.intercept_[0])

# without frame-change

In [ ]:
fig = ThreeDFigure()
fig.add_markers(X, color=y) #samples

xx, yy = make_meshgrid(X.min())                
fig.add_surface(xx, yy, decision_plane.z) #decision hyperplane
fig.add_plane(decision_plane)

fig.add_line(X.mean(axis=0)-decision_plane.normal, X.mean(axis=0)+decision_plane.normal) #orthogonal of decision hyperplane through mean of points
fig.add_markers([0,0,0], size=3) #coordinate center

fig.show()

# with frame-change

In [ ]:
forward, backward = make_base_changer(decision_plane)

fig = ThreeDFigure(forward, backward)

fig.add_markers(X, color=y) #samples

xx, yy = make_meshgrid(X.min())                
fig.add_surface(xx, yy, decision_plane.z) #decision hyperplane

fig.add_line(X.mean(axis=0)-decision_plane.normal, X.mean(axis=0)+decision_plane.normal) #orthogonal of decision hyperplane through mean of points
fig.add_markers([0,0,0], size=3) #coordinate center

fig.add_plane(decision_plane)

fig.show()

In [ ]:
transformed_normal = forward(model.coef_[0])
transformed_orig = forward([0,0,0])
new_plane = Plane(*(transformed_normal-transformed_orig), 0)

new_plane

In [ ]:
fig = ThreeDFigure(forward, backward)

fig.add_markers(X, color=y) #samples

#xx, yy = make_meshgrid(X.min())                
#fig.add_surface(xx, yy, new_plane.z) #decision hyperplane

fig.add_line(X.mean(axis=0)-decision_plane.normal, X.mean(axis=0)+decision_plane.normal) #orthogonal of decision hyperplane through mean of points
fig.add_markers([0,0,0], size=3) #coordinate center

fig.add_plane(new_plane)

fig.show()